In [34]:
import cv2
from yolo_segmentation import YOLOSegmentation
import numpy as np

In [35]:
def rszImg(image, rate):
    h, w, _ = image.shape
    new_h, new_w = (int(h*rate), int(w*rate))
    image = cv2.resize(image, (new_w, new_h))
    return image

def cropPoly(img, bbox, seg):
    (x, y, x2, y2) = bbox
    # Create a mask of the same size as the image
    mask = np.zeros_like(img)

    # Convert the seg (polygon) into a NumPy array of type int32
    points = np.array(seg, dtype=np.int32)

    # Fill the mask with a white polygon based on the points
    cv2.fillPoly(mask, [points], (255, 255, 255))

    # Bitwise AND the mask with the original image to get the cropped region
    cropped_motorcycle = cv2.bitwise_and(img, mask)

    # Crop the region inside the bounding box
    cropped_motorcycle = cropped_motorcycle[y:y2, x:x2]
    
    return cropped_motorcycle

def segmentMotorcycle(path):
    global ys
    
    img = cv2.imread(path)
    img = rszImg(img, 0.25)
    bboxes, classes, class_names, segmentation, scores = ys.detect(img)

    cropped_motorcycle_regions = []
    n = 0

    for bbox, class_id, seg, score in zip(bboxes, classes, segmentation, scores):
        # print("bbox:", bbox, " class_id:", class_id, " seg:", seg, " score:", score)
        if class_names[class_id] == 'motorcycle':
            (x, y, x2, y2) = bbox

            cropped_motorcycle = cropPoly(img, bbox, seg)
            cropped_motorcycle_regions.append(cropped_motorcycle)

            # Draw the bounding box, polyline, and class name on the original image
            cv2.rectangle(img, (x, y), (x2, y2), (0, 0, 255), 2)
            cv2.polylines(img, [seg], True, (255, 0, 0), 2)
            cv2.putText(img, str(class_names[class_id]), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
            
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [36]:
global ys
ys = YOLOSegmentation('../learn/yolov8-polylines/yolov8n-seg.pt') 

In [37]:

path_nyala = '../dataset-baru/scoopy-mati/52.jpg'
# path_nyala = 'awikwok.jpg'
path_mati = '../dataset-baru/beat-mati/0.jpg'


# flag = 'nyala'
# img = cv2.imread(path_nyala)
flag = 'mati'
img = cv2.imread(path_nyala)

# img = cv2.resize(img, None, fx=0.2, fy=0.2)
img = rszImg(img, 0.25)

In [38]:
# Segmentation
# ys = YOLOSegmentation('../../yolov8n-seg.pt')
bboxes, classes, class_names, segmentation, scores = ys.detect(img)
print(scores)


0: 640x480 2 persons, 1 car, 1 motorcycle, 515.7ms
Speed: 6.0ms preprocess, 515.7ms inference, 20.6ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.


[       0.85        0.33        0.32        0.31]


In [39]:
cropped_motorcycle_regions = []
n = 0

for bbox, class_id, seg, score in zip(bboxes, classes, segmentation, scores):
    # print("bbox:", bbox, " class_id:", class_id, " seg:", seg, " score:", score)
    if class_names[class_id] == 'motorcycle':
        (x, y, x2, y2) = bbox

        cropped_motorcycle = cropPoly(img, bbox, seg)
        cropped_motorcycle_regions.append(cropped_motorcycle)

        # Draw the bounding box, polyline, and class name on the original image
        cv2.rectangle(img, (x, y), (x2, y2), (0, 0, 255), 2)
        cv2.polylines(img, [seg], True, (255, 0, 0), 2)
        cv2.putText(img, str(class_names[class_id]), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)

In [40]:
print(cropped_motorcycle_regions[0])

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [41]:
# Save the cropped motorcycle regions or do further processing
for i, cropped_motorcycle in enumerate(cropped_motorcycle_regions):
    if flag == 'nyala':
        cv2.imwrite(f'result/nyala/{i}.jpg', cropped_motorcycle)
    else :
        cv2.imwrite(f'result/mati/{i}.jpg', cropped_motorcycle)
        

In [42]:
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [43]:
segmentMotorcycle(path_nyala)


0: 640x480 2 persons, 1 car, 1 motorcycle, 547.3ms
Speed: 10.1ms preprocess, 547.3ms inference, 21.2ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.
